## Imports

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold, train_test_split

dir_parts = os.getcwd().split(os.path.sep)
root_index = dir_parts.index('ML-B')
root_path = os.path.sep.join(dir_parts[:root_index + 1])
sys.path.append(root_path + '/code/')
from data.data_config import Dataset
from data.data_utils import load_monk, load_cup, store_monk_result, store_cup_result
from hyperparameter_tuning import tuning_search_top_configs
from training.metrics import mean_euclidean_error

%load_ext autoreload
%autoreload 2

# Random Forest
In this notebook we test a **RandomForestClassifier** and a **RandomForestRegressor** w.r.t. the tasks at hand, i.e. the three MONK's problems and the CUP dataset respectively.

In [ ]:
MODEL_NAME = 'RandomForest'
INTERNAL_TEST_SPLIT = 0.1 # internal test split percentage
RANDOM_STATE = 128 # reproducibility
N_SPLITS = 5 # cross-validation

## Path

In [ ]:
# Directories
results_dir = root_path + '/results/' + MODEL_NAME

# Filepaths (MONK)
m1_dev_path, m1_test_path = Dataset.MONK_1.dev_path, Dataset.MONK_1.test_path # MONK 1
m2_dev_path, m2_test_path = Dataset.MONK_2.dev_path, Dataset.MONK_2.test_path # MONK 2
m3_dev_path, m3_test_path = Dataset.MONK_3.dev_path, Dataset.MONK_3.test_path # MONK 3

# Filepaths (CUP)
cup_dev_path, cup_test_path = Dataset.CUP.dev_path, Dataset.CUP.test_path

# MONK-1

In [ ]:
# Load MONK-1
x_dev_m1, y_dev_m1, x_test_m1, y_test_m1 = load_monk(m1_dev_path, m1_test_path)

## Grid Search
Grid search is a simple hyper-parameter tuning techniques useful for finding the best configuration for a specific ML model. It involves:
- defining a grid of hparams values
- systematically test all possible combinations

Other approaches include Random search or Bayesian optimization.

In [ ]:
# hparams grid for grid search
rfc_hparams_spaces = {
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 4],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [False],
}

In [ ]:
# perform grid search with KFold
grid_search_m1 = GridSearchCV(
    RandomForestClassifier(random_state=128),
    param_grid=rfc_hparams_spaces,
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=128),
    scoring='accuracy',
    verbose=1
)

grid_search_m1.fit(x_dev_m1, y_dev_m1)

In [ ]:
best_configs_m1 = tuning_search_top_configs(grid_search_m1.cv_results_) # top k config

## Training - Testing

In [ ]:
# Create and train the RandomForestClassifier
rfc_m1 = RandomForestClassifier(**grid_search_m1.best_params_)
rfc_m1.fit(x_dev_m1, y_dev_m1)

In [ ]:
print('-- DEVELOPMENT --')
acc_dev_m1 = accuracy_score(y_dev_m1, rfc_m1.predict(x_dev_m1))
mse_dev_m1 = mean_squared_error(y_dev_m1, rfc_m1.predict(x_dev_m1))
print(f'MSE: {mse_dev_m1:.4f} - Accuracy: {acc_dev_m1:.4f}')

In [ ]:
print('-- TEST --')
acc_test_m1 = accuracy_score(y_test_m1, rfc_m1.predict(x_test_m1))
mse_test_m1 = mean_squared_error(y_test_m1, rfc_m1.predict(x_test_m1))
print(f'MSE: {mse_test_m1:.4f} - Accuracy: {acc_test_m1:.4f}')

## Store results

In [ ]:
report_m1 = {
    'dev': {'mse': mse_dev_m1, 'accuracy': acc_dev_m1},
    'test': {'mse': mse_test_m1, 'accuracy': acc_test_m1}
}

store_monk_result(results_dir + '/MONK1/', grid_search_m1.best_params_, report_m1)

# MONK-2

In [ ]:
# Load MONK-2
x_dev_m2, y_dev_m2, x_test_m2, y_test_m2 = load_monk(m2_dev_path, m2_test_path)

## Grid Search

In [ ]:
# perform grid search with KFold
grid_search_m2 = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE),
    param_grid=rfc_hparams_spaces,
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE),
    scoring='accuracy',
    verbose=1
)

grid_search_m2.fit(x_dev_m2, y_dev_m2)

In [ ]:
best_configs_m2 = tuning_search_top_configs(grid_search_m2.cv_results_) # top k config

## Training - Testing

In [ ]:
# Create and train the RandomForestClassifier
rfc_m2 = RandomForestClassifier(**grid_search_m2.best_params_)
rfc_m2.fit(x_dev_m2, y_dev_m2)

In [ ]:
print('-- DEVELOPMENT --')
acc_dev_m2 = accuracy_score(y_dev_m2, rfc_m2.predict(x_dev_m2))
mse_dev_m2 = mean_squared_error(y_dev_m2, rfc_m2.predict(x_dev_m2))
print(f'MSE: {mse_dev_m2:.4f} - Accuracy: {acc_dev_m2:.4f}')

In [ ]:
print('-- TEST --')
acc_test_m2 = accuracy_score(y_test_m2, rfc_m2.predict(x_test_m2))
mse_test_m2 = mean_squared_error(y_test_m2, rfc_m2.predict(x_test_m2))
print(f'MSE: {mse_test_m2:.4f} - Accuracy: {acc_test_m2:.4f}')

## Store results

In [ ]:
report_m2 = {
    'dev': {'mse': mse_dev_m2, 'accuracy': acc_dev_m2},
    'test': {'mse': mse_test_m2, 'accuracy': acc_test_m2}
}

store_monk_result(results_dir + '/MONK2/', grid_search_m2.best_params_, report_m2)

# MONK-3

In [ ]:
# Load MONK-3
x_dev_m3, y_dev_m3, x_test_m3, y_test_m3 = load_monk(m3_dev_path, m3_test_path)

## Grid Search

In [ ]:
# perform grid search with KFold
grid_search_m3 = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE),
    param_grid=rfc_hparams_spaces,
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE),
    scoring='accuracy',
    verbose=1
)

grid_search_m3.fit(x_dev_m3, y_dev_m3)

In [ ]:
best_configs_m3 = tuning_search_top_configs(grid_search_m3.cv_results_) # top k config

## Training - Testing

In [ ]:
# Create and train the RandomForestClassifier
rfc_m3 = RandomForestClassifier(**grid_search_m3.best_params_)
rfc_m3.fit(x_dev_m3, y_dev_m3)

In [ ]:
print('-- DEVELOPMENT --')
acc_dev_m3 = accuracy_score(y_dev_m3, rfc_m3.predict(x_dev_m3))
mse_dev_m3 = mean_squared_error(y_dev_m3, rfc_m3.predict(x_dev_m3))
print(f'MSE: {mse_dev_m3:.4f} - Accuracy: {acc_dev_m3:.4f}')

In [ ]:
print('-- TEST --')
acc_test_m3 = accuracy_score(y_test_m3, rfc_m3.predict(x_test_m3))
mse_test_m3 = mean_squared_error(y_test_m3, rfc_m3.predict(x_test_m3))
print(f'MSE: {mse_test_m3:.4f} - Accuracy: {acc_test_m3:.4f}')

## Store results

In [ ]:
report_m3 = {
    'dev': {'mse': mse_dev_m3, 'accuracy': acc_dev_m3},
    'test': {'mse': mse_test_m3, 'accuracy': acc_test_m3}
}

store_monk_result(results_dir + '/MONK3/', grid_search_m3.best_params_, report_m3)

# CUP

In [ ]:
# Load CUP
x_dev_cup, y_dev_cup, x_test_cup = load_cup(cup_dev_path, cup_test_path)

## Dev - Internal Test Split 
The development dataset is split between training and internal test ($90-10$).

In [ ]:
# Split the internal test set
x_train_cup, x_internal_test_cup, y_train_cup, y_internal_test_cup = train_test_split(
    x_dev_cup, y_dev_cup, test_size=0.1, random_state=128
)

## Hyper-paramater Tuning

### Grid Search

In [ ]:
rfr_hparams_spaces_gridsearch = {
    'n_estimators': [100, 200],
    'max_features': ['sqrt'],
    'max_depth': [5, 10, 20, 50, None],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['squared_error'],
    'bootstrap': [False]
}

In [ ]:
grid_search_cup = GridSearchCV(
    RandomForestRegressor(random_state=RANDOM_STATE), 
    rfr_hparams_spaces_gridsearch, 
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE),
    scoring=make_scorer(mean_euclidean_error, greater_is_better=False),
    verbose=1
)

grid_search_cup.fit(x_train_cup, y_train_cup)

In [ ]:
best_configs_cup = tuning_search_top_configs(grid_search_cup.cv_results_) # top k config

### Random Search

In [ ]:
best_params = grid_search_cup.best_params_
print(best_params)

In [ ]:
# Define new parameter space for RandomizedSearchCV around the best parameters
rfr_hparams_spaces_randomsearch = {
    'n_estimators': randint(best_params['n_estimators'] - 50, best_params['n_estimators'] + 50),
    'max_features': [best_params['max_features']],
    'max_depth': [best_params['max_depth']],
    'min_samples_split': randint(max(2, best_params['min_samples_split'] - 2), best_params['min_samples_split'] + 3),
    'min_samples_leaf': randint(max(1, best_params['min_samples_leaf'] - 1), best_params['min_samples_leaf'] + 2),
    'criterion': [best_params['criterion']],
    'bootstrap': [best_params['bootstrap']]
}

In [ ]:
random_search_cup = RandomizedSearchCV(
    RandomForestRegressor(random_state=RANDOM_STATE), 
    rfr_hparams_spaces_randomsearch, 
    n_iter=5, # number of configs sampled (do >100)
    cv=KFold(n_splits=2, shuffle=True, random_state=RANDOM_STATE),
    scoring=make_scorer(mean_euclidean_error, greater_is_better=False),
    verbose=1,
    random_state=RANDOM_STATE
)

random_search_cup.fit(x_train_cup, y_train_cup)

In [ ]:
best_score_grid = grid_search_cup.best_score_
best_score_random = random_search_cup.best_score_

# Check if best result is from GridSearc or RandomSearch
if best_score_random > best_score_grid:
    print("Best configuration from RandomizedSearch:\n")
    best_hparams = random_search_cup.best_params_
    print(best_hparams)
else:
    print("GridSearchCV resulted in the best configuration.")

## Training - Testing

In [ ]:
# Create a RandomForestRegressor with the best hparams
rfr = RandomForestRegressor(**best_params)

# Train the model
rfr.fit(x_train_cup, y_train_cup)

In [ ]:
print('-- TRAINING --')
mee_train_cup = mean_euclidean_error(y_train_cup, rfr.predict(x_train_cup))
mse_train_cup = mean_squared_error(y_train_cup, rfr.predict(x_train_cup))
print(f'Loss (MSE): {mse_train_cup:.4f} - MEE: {mee_train_cup:.4f}')

In [ ]:
print('-- INTERNAL TEST --')
mee_internal_test_cup = mean_euclidean_error(y_internal_test_cup, rfr.predict(x_internal_test_cup))
mse_internal_test_cup = mean_squared_error(y_internal_test_cup, rfr.predict(x_internal_test_cup))
print(f'Loss (MSE): {mse_internal_test_cup:.4f} - MEE: {mee_internal_test_cup:.4f}')

In [ ]:
# Blind test set predictions
blind_test_preds_cup = rfr.predict(x_test_cup)

## Store results

In [ ]:
best_index = grid_search_cup.best_index_
mee_mean_score = -grid_search_cup.cv_results_['mean_test_score'][best_index]
mee_std_dev = std_validation_error = grid_search_cup.cv_results_['std_test_score'][best_index]

report_cup = {
    'train': {'mse': mse_train_cup, 'mee': mee_train_cup},
    'validation': {'mean_mee': mee_mean_score, 'std_dev_mee': mee_std_dev},
    'internal_test': {'mse': mse_internal_test_cup, 'mee': mee_internal_test_cup},
}

store_cup_result(results_dir + '/CUP/', grid_search_cup.best_params_, report_cup, blind_test_preds_cup)